In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langgraph.graph import START, END, StateGraph
from typing_extensions import TypedDict
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from collections.abc import Sequence

In [3]:
# defining the state

class State(TypedDict):
    messages : Sequence[BaseMessage]

state = State(messages = [HumanMessage("Tell me a joke!")])    

In [4]:
# defining the openai chat api

chat = ChatOpenAI(
    model = "gpt-4o",
    seed = 365,
    temperature= 0,
    max_completion_tokens=100
)

In [ ]:
# Node 1 Ask a question

def Node1(state: State) -> State:
    print("Entering Node 1 ---------------------- ")

    user_input = input()

    return State(messages=[HumanMessage(user_input)])

In [ ]:
# Node 2 Get a response from the openai model

def Node2(state: State) -> State:

    print("Entering node 2---------------------------------")
    print("Getting a response from the model...............")

    response = chat.invoke(state['messages'])

    response.pretty_print()

    return State(messages=[response]) 

In [ ]:
# Node 3 Ask a question again

def Node3(state: State) -> State:
    print("Entering Node 3 ---------------------- ")
    print("Would you like to ask me a question again ---------------------- ")

    user_input = input()

    return State(messages=[HumanMessage(user_input)])

In [22]:
# Build the graph

graph = StateGraph(State)

In [ ]:
# define a path to decide between states
def path_decider(state: State) -> str:
    if state['messages'][0].content == "yes":
        return "True"
    else:
        return "False"    

In [23]:
# Add the nodes

graph.add_node("Node1", Node1)
graph.add_node("Node2", Node2)
graph.add_node("Node3", Node3)

# Add the edges

graph.add_edge(START, "Node1")
graph.add_edge("Node1", "Node2")
graph.add_edge("Node2", "Node3")

# Adding conditional edge to node 3 as user can input or end the session

graph.add_conditional_edges(
    source = "Node3",
    path = path_decider,
    path_map = {
        "True" : "Node1",
        "False" : END
    }
    )

In [18]:
graph_compiled = graph.compile()

In [24]:
graph_compiled.invoke(State(messages = []))

Entering Node 1 ---------------------- 
Entering node 2---------------------------------
Getting a response from the model...............
================================== Ai Message ==================================

I'm sorry, but I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. If you have any questions or need information on a specific topic, feel free to ask!
Entering Node 3 ---------------------- 
Would you like to ask me a question again ---------------------- 


KeyError: 'False'

In [ ]:
graph_compiled.get_graph().draw_ascii()

In [ ]:
# define a path to decide between states using Literal data type
from typing import Literal


def path_decider(state: state) -> Literal["Node1", END]:
    if state['messages'][0].content == "yes":
        return "Node1"
    else:
        return END   

In [ ]:
# Add the nodes

graph.add_node("Node1", Node1)
graph.add_node("Node2", Node2)
graph.add_node("Node3", Node3)

# Add the edges

graph.add_edge(START, "Node1")
graph.add_edge("Node1", "Node2")
graph.add_edge("Node2", "Node3")

# Adding conditional edge to node 3 as user can input or end the session, without the path map

graph.add_conditional_edges(
    source = "Node3",
    path = path_decider,
    )